In [1]:
import torch
from torch import nn
import torch.nn.functional as F

import math
from inspect import isfunction

# constants

MIN_EXPERT_CAPACITY = 4

# helper functions

def default(val, default_val):
    default_val = default_val() if isfunction(default_val) else default_val
    return val if val is not None else default_val

def cast_tuple(el):
    return el if isinstance(el, tuple) else (el,)

# tensor related helper functions

def top1(t):
    # 最后一维的第一大
    values, index = t.topk(k=1, dim=-1)
    # map(function,iterable)map() 会根据提供的函数对指定序列做映射。
    # 第一个参数 function 以参数序列中的每一个元素调用 function 函数，返回包含每次 function 函数返回值的新列表。
    # squeeze去掉dim维，这一维的维度必须为1，相当于降维，去掉维度为1的没用的维度
    values, index = map(lambda x: x.squeeze(dim=-1), (values, index))
    return values, index

def cumsum_exclusive(t, dim=-1):
    num_dims = len(t.shape)
    # 算出dim后面有几维，因为pad的时候是从后往前，dim后面的不扩充，只有dim向上扩充一维
    num_pad_dims = - dim - 1
    pre_padding = (0, 0) * num_pad_dims
    # slice(stop)/slice(start,stop[,step]) 切片函数，slice(5)切[0,5)，slice(1,5)切[1,5)
    pre_slice   = (slice(None),) * num_pad_dims
    # cumsum 根据指定的维度从前向后累加（不是sum，返回的是[a1,a2+a1,a3+a2+a1,……]）
    # pad填充输入 F.pad(t,(0,0,1,0))意思是最后一维向左扩充0维，向右扩充0维；倒数第二维向上扩充1维，向下扩充0维。以此类推。
    # pre_padding的作用体现在这里，从后往前扩充，没到dim都不扩充，到dim了往上走一维
    padded_t = F.pad(t, (*pre_padding, 1, 0)).cumsum(dim=dim)
    # 切掉dim的最后一维度。上面扩充了0，下面为什么要把总和切了？
    return padded_t[(..., slice(None, -1), *pre_slice)]

# pytorch one hot throws an error if there are out of bound indices.
# tensorflow, in contrast, does not throw an error
def safe_one_hot(indexes, max_length):
    max_index = indexes.max() + 1
    return F.one_hot(indexes, max(max_index + 1, max_length))[..., :max_length]

def init_(t):
    dim = t.shape[-1]
    std = 1 / math.sqrt(dim)
    return t.uniform_(-std, std)

# activations

class GELU_(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(math.sqrt(2 / math.pi) * (x + 0.044715 * torch.pow(x, 3))))

GELU = nn.GELU if hasattr(nn, 'GELU') else GELU_

# expert class

class Experts(nn.Module):
    def __init__(self,
        dim,
        num_experts = 16,
        hidden_dim = None,
        activation = GELU):
        super().__init__()

        hidden_dim = default(hidden_dim, dim * 4)
        num_experts = cast_tuple(num_experts)

        w1 = torch.zeros(*num_experts, dim, hidden_dim)
        w2 = torch.zeros(*num_experts, hidden_dim, dim)

        w1 = init_(w1)
        w2 = init_(w2)

        self.w1 = nn.Parameter(w1)
        self.w2 = nn.Parameter(w2)
        self.act = activation()

    def forward(self, x):
        hidden = torch.einsum('...nd,...dh->...nh', x, self.w1)
        hidden = self.act(hidden)
        out    = torch.einsum('...nh,...hd->...nd', hidden, self.w2)
        return out

# the below code is almost all transcribed from the official tensorflow version, from which the papers are written
# https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/research/moe.py

# gating network

class Top2Gating(nn.Module):
    def __init__(
        self,
        dim,
        num_gates,
        eps = 1e-9,
        outer_expert_dims = tuple(),
        second_policy_train = 'random',
        second_policy_eval = 'random',
        second_threshold_train = 0.2,
        second_threshold_eval = 0.2,
        capacity_factor_train = 1.25,
        capacity_factor_eval = 2.):
        super().__init__()

        self.eps = eps
        self.num_gates = num_gates
        # w_gating维度是 outer_expert_dims * dim * num_gates
        # 输出是门数，所以最后一维是num_gates
        # dim是特征维数，作为中间维度与输入相乘
        # outer_experts_dims是啥？
        self.w_gating = nn.Parameter(torch.randn(*outer_expert_dims, dim, num_gates))

        self.second_policy_train = second_policy_train
        self.second_policy_eval = second_policy_eval
        self.second_threshold_train = second_threshold_train
        self.second_threshold_eval = second_threshold_eval
        self.capacity_factor_train = capacity_factor_train
        self.capacity_factor_eval = capacity_factor_eval

    def forward(self, x, importance = None):
        *_, b, group_size, dim = x.shape
        num_gates = self.num_gates

        # 这应该是module自带参数，train（）的时候为True，eval（）的时候为负
        if self.training:
            policy = self.second_policy_train
            threshold = self.second_threshold_train
            capacity_factor = self.capacity_factor_train
        else:
            policy = self.second_policy_eval
            threshold = self.second_threshold_eval
            capacity_factor = self.capacity_factor_eval

        raw_gates = torch.einsum('...bnd,...de->...bne', x, self.w_gating)
        # 最后一维是gate，所以按照最后一维softmax
        raw_gates = raw_gates.softmax(dim=-1)

        # FIND TOP 2 EXPERTS PER POSITON
        # Find the top expert for each position. shape=[batch, group]

        gate_1, index_1 = top1(raw_gates)
        mask_1 = F.one_hot(index_1, num_gates).float()
        density_1_proxy = raw_gates
        
        # 默认不用L_importance
        if importance is not None:
            equals_one_mask = (importance == 1.).float()
            mask_1 *= equals_one_mask[..., None]
            gate_1 *= equals_one_mask
            density_1_proxy *= equals_one_mask[..., None]
            del equals_one_mask

        # mask_1是top1下标的one-hot编码，1-mask_1也就意味着top1对应的位置*0，其他的位置*1
        # 变成0了就是最小的了，再一次top1就可以提取出top2了    
        gates_without_top_1 = raw_gates * (1. - mask_1)

        gate_2, index_2 = top1(gates_without_top_1)
        mask_2 = F.one_hot(index_2, num_gates).float()

        if importance is not None:
            greater_zero_mask = (importance > 0.).float()
            mask_2 *= greater_zero_mask[..., None]
            del greater_zero_mask

        # normalize top2 gate scores
        #只保留最大的2个，这里相当于对最大的两个又进行一次softmax
        denom = gate_1 + gate_2 + self.eps
        gate_1 /= denom
        gate_2 /= denom

        # BALANCING LOSSES
        # shape = [batch, experts]
        # We want to equalize the fraction of the batch assigned to each expert
        density_1 = mask_1.mean(dim=-2) # [batch_size, expert_num]每个expert在每个batch里被选中的平均次数
        # Something continuous that is correlated with what we want to equalize.
        # 没看懂怎么算的
        density_1_proxy = density_1_proxy.mean(dim=-2) # [batch_size, expert_num]每个expert在每个batch里softmax后平均数值
        loss = (density_1_proxy * density_1).mean() * float(num_gates ** 2)

        # Depending on the policy in the hparams, we may drop out some of the
        # second-place experts.
        # 对第二expert进行类似dropout的操作，可能删去低于阈值的，可能随机扔一些，可能都不要，可能都要
        if policy == "all":
            pass
        elif policy == "none":
            mask_2 = torch.zeros_like(mask_2)
        elif policy == "threshold":
            mask_2 *= (gate_2 > threshold).float()
        elif policy == "random":
            # uniform_在均匀分布中随机采样，左闭右开
            probs = torch.zeros_like(gate_2).uniform_(0., 1.) #[inputs[0], outer_expert_dim]
            # 是mask_2*=，不是=，unsqueeze加一维
            mask_2 *= (probs < (gate_2 / max(threshold, self.eps))).float().unsqueeze(-1)
        else:
            raise ValueError(f"Unknown policy {policy}")

        # Each sequence sends (at most?) expert_capacity positions to each expert.
        # Static expert_capacity dimension is needed for expert batch sizes
        expert_capacity = min(group_size, int((group_size * capacity_factor) / num_gates))
        expert_capacity = max(expert_capacity, MIN_EXPERT_CAPACITY)
        expert_capacity_f = float(expert_capacity)

        # COMPUTE ASSIGNMENT TO EXPERTS
        # [batch, group, experts]
        # This is the position within the expert's mini-batch for this sequence
        position_in_expert_1 = cumsum_exclusive(mask_1, dim=-2) * mask_1
        # Remove the elements that don't fit. [batch, group, experts]
        mask_1 *= (position_in_expert_1 < expert_capacity_f).float()
        # [batch, experts]
        # How many examples in this sequence go to this expert
        mask_1_count = mask_1.sum(dim=-2, keepdim=True)
        # [batch, group] - mostly ones, but zeros where something didn't fit
        mask_1_flat = mask_1.sum(dim=-1)
        # [batch, group]
        position_in_expert_1 = position_in_expert_1.sum(dim=-1)
        # Weight assigned to first expert.  [batch, group]
        gate_1 *= mask_1_flat

        # 加上mask_1_count,是算每个expert一共被加了几次，不能超过expert_capacity
        position_in_expert_2 = cumsum_exclusive(mask_2, dim=-2) + mask_1_count
        position_in_expert_2 *= mask_2
        mask_2 *= (position_in_expert_2 < expert_capacity_f).float()
        mask_2_flat = mask_2.sum(dim=-1)

        position_in_expert_2 = position_in_expert_2.sum(dim=-1)
        gate_2 *= mask_2_flat
        
        # [batch, group, experts, expert_capacity]
        # None就是这一维的里面加一维
        # F.one_hot及之前的累乘[batch,group,expert,1],safe_one_hot是[batch,group,1,expert_capacity]
        combine_tensor = (
            gate_1[..., None, None]
            * mask_1_flat[..., None, None]
            * F.one_hot(index_1, num_gates)[..., None]
            * safe_one_hot(position_in_expert_1.long(), expert_capacity)[..., None, :] +
            gate_2[..., None, None]
            * mask_2_flat[..., None, None]
            * F.one_hot(index_2, num_gates)[..., None]
            * safe_one_hot(position_in_expert_2.long(), expert_capacity)[..., None, :]
        )

        dispatch_tensor = combine_tensor.bool().to(combine_tensor)
        return dispatch_tensor, combine_tensor, loss

# plain mixture of experts

class MoE(nn.Module):
    def __init__(self,
        dim,
        num_experts = 16,
        hidden_dim = None,
        activation = nn.ReLU,
        second_policy_train = 'random',
        second_policy_eval = 'random',
        second_threshold_train = 0.2,
        second_threshold_eval = 0.2,
        capacity_factor_train = 1.25,
        capacity_factor_eval = 2.,
        loss_coef = 1e-2,
        experts = None):
        super().__init__()

        self.num_experts = num_experts

        gating_kwargs = {'second_policy_train': second_policy_train, 'second_policy_eval': second_policy_eval, 'second_threshold_train': second_threshold_train, 'second_threshold_eval': second_threshold_eval, 'capacity_factor_train': capacity_factor_train, 'capacity_factor_eval': capacity_factor_eval}
        # 初始化gate，传入门的数量，和超参，没有传入outer_expert_dims
        self.gate = Top2Gating(dim, num_gates = num_experts, **gating_kwargs)
        self.experts = default(experts, lambda: Experts(dim, num_experts = num_experts, hidden_dim = hidden_dim, activation = activation))
        self.loss_coef = loss_coef

    def forward(self, inputs, **kwargs):
        b, n, d, e = *inputs.shape, self.num_experts
        dispatch_tensor, combine_tensor, loss = self.gate(inputs)
        expert_inputs = torch.einsum('bnd,bnec->ebcd', inputs, dispatch_tensor)

        # Now feed the expert inputs through the experts.
        orig_shape = expert_inputs.shape
        expert_inputs = expert_inputs.reshape(e, -1, d)
        expert_outputs = self.experts(expert_inputs)
        expert_outputs = expert_outputs.reshape(*orig_shape)

        output = torch.einsum('ebcd,bnec->bnd', expert_outputs, combine_tensor)
        return output, loss * self.loss_coef


In [1]:
import torch
from torch import nn
# from mixture_of_experts import MoE

moe = MoE(
    dim = 512,
    num_experts = 16,               # increase the experts (# parameters) of your model without increasing computation
    hidden_dim = 512 * 4,           # size of hidden dimension in each expert, defaults to 4 * dimension
    activation = nn.LeakyReLU,      # use your preferred activation, will default to GELU
    second_policy_train = 'random', # in top_2 gating, policy for whether to use a second-place expert
    second_policy_eval = 'random',  # all (always) | none (never) | threshold (if gate value > the given threshold) | random (if gate value > threshold * random_uniform(0, 1))
    second_threshold_train = 0.2,
    second_threshold_eval = 0.2,
    capacity_factor_train = 1.25,   # experts have fixed capacity per batch. we need some extra capacity in case gating is not perfectly balanced.
    capacity_factor_eval = 2.,      # capacity_factor_* should be set to a value >=1
    loss_coef = 1e-2                # multiplier on the auxiliary expert balancing auxiliary loss
)

inputs = torch.randn(4, 1024, 512)
out, aux_loss = moe(inputs)

In [3]:
outer_expert_dims=tuple([3])
dim=4
num_gates=5
w_gating = nn.Parameter(torch.randn(*outer_expert_dims, dim, num_gates))
print(w_gating)
# 将一个不可训练的类型Tensor转换成可以训练的类型parameter并将这个parameter绑定到这个module里面
# (net.parameter()中就有这个绑定的parameter，所以在参数优化的时候可以进行优化的)
# 所以经过类型转换变成了模型的一部分，成为了模型中根据训练可以改动的参数了
# 使用这个函数的目的也是想让某些变量在学习的过程中不断的修改其值以达到最优化。

Parameter containing:
tensor([[[-0.9714, -0.0394, -0.1725, -0.9955, -0.4789],
         [ 0.2099,  0.3940, -0.6335,  2.1337, -0.3239],
         [-1.1338, -1.1216, -0.1250,  0.1894, -0.6138],
         [-0.4100, -1.2906,  2.0499,  2.3571,  0.1728]],

        [[ 0.3384, -0.2154, -0.0921,  0.9377, -1.2150],
         [-0.3789,  0.1804,  0.9904, -0.3555, -0.4614],
         [ 2.1593,  0.1664,  1.2265,  0.4760,  0.5301],
         [ 0.7857, -0.1060,  0.7419,  0.5517, -0.1416]],

        [[ 1.0148,  2.0236,  0.7502, -2.2973,  0.0156],
         [-0.8988,  0.4086, -0.4696, -0.5865,  0.2939],
         [ 1.4902, -0.0257,  0.1126,  0.8801, -0.9802],
         [-1.9549,  2.3498, -0.0437, -0.0219, -0.6894]]], requires_grad=True)


In [4]:
inputs=torch.randn(3,4)
print("inputs=",inputs)
# *_, b, group_size, dim = inputs.shape
# print("_,b,group_size,dim=",_,b,group_size,dim)
raw_gates = torch.einsum('ad,bde->abe', inputs, w_gating)
print("raw_gates=",raw_gates)
raw_gates = raw_gates.softmax(dim=-1)
print("raw_gates=",raw_gates)

inputs= tensor([[-0.9795,  0.1923, -0.6336,  0.3192],
        [ 0.5834,  0.5061,  0.9854, -0.9274],
        [-1.9581,  1.6119,  0.5217,  0.2567]])
raw_gates= tensor([[[ 1.5793,  0.4130,  0.7807,  2.0178,  0.8509],
         [-1.5216,  0.1064, -0.2597, -1.1123,  0.7202],
         [-2.7351, -1.1371, -0.9104,  1.5727,  0.4423]],

        [[-1.1975,  0.2681, -2.4454, -1.5001, -1.2085],
         [ 1.4049,  0.2280,  0.9682,  0.3245, -0.2886],
         [ 3.4187, -0.8172,  0.3514, -0.7495, -0.1687]],

        [[ 1.5438, -0.2042, -0.2223,  6.0924,  0.1399],
         [ 0.0548,  0.7722,  2.6070, -2.0191,  1.8755],
         [-3.1603, -2.7140, -2.1785,  4.0064, -0.2450]]],
       grad_fn=<ViewBackward0>)
raw_gates= tensor([[[2.6355e-01, 8.2095e-02, 1.1859e-01, 4.0857e-01, 1.2720e-01],
         [4.8679e-02, 2.4796e-01, 1.7195e-01, 7.3303e-02, 4.5811e-01],
         [9.0573e-03, 4.4773e-02, 5.6165e-02, 6.7278e-01, 2.1722e-01]],

        [[1.3615e-01, 5.8951e-01, 3.9086e-02, 1.0060e-01, 1.3466e-01],
   

In [5]:
gate_1,index_1=top1(raw_gates)

In [6]:
print(gate_1,index_1)

tensor([[0.4086, 0.4581, 0.6728],
        [0.5895, 0.4036, 0.9057],
        [0.9834, 0.5785, 0.9820]], grad_fn=<SqueezeBackward1>) tensor([[3, 4, 3],
        [1, 0, 0],
        [3, 2, 3]])


In [7]:
values, index = raw_gates.topk(k=1, dim=-1)
print(values, index)
values, index = map(lambda x: x.squeeze(dim=-1), (values, index))
print(values, index)

tensor([[[0.4086],
         [0.4581],
         [0.6728]],

        [[0.5895],
         [0.4036],
         [0.9057]],

        [[0.9834],
         [0.5785],
         [0.9820]]], grad_fn=<TopkBackward0>) tensor([[[3],
         [4],
         [3]],

        [[1],
         [0],
         [0]],

        [[3],
         [2],
         [3]]])
tensor([[0.4086, 0.4581, 0.6728],
        [0.5895, 0.4036, 0.9057],
        [0.9834, 0.5785, 0.9820]], grad_fn=<SqueezeBackward1>) tensor([[3, 4, 3],
        [1, 0, 0],
        [3, 2, 3]])


In [8]:
mask_1 = F.one_hot(index_1, num_gates).float()
density_1_proxy = raw_gates
print(mask_1)
gates_without_top_1 = raw_gates * (1. - mask_1)
print(gates_without_top_1)
gate_2, index_2 = top1(gates_without_top_1)
mask_2 = F.one_hot(index_2, num_gates).float()

tensor([[[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.]],

        [[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.]]])
tensor([[[0.2635, 0.0821, 0.1186, 0.0000, 0.1272],
         [0.0487, 0.2480, 0.1719, 0.0733, 0.0000],
         [0.0091, 0.0448, 0.0562, 0.0000, 0.2172]],

        [[0.1361, 0.0000, 0.0391, 0.1006, 0.1347],
         [0.0000, 0.1244, 0.2608, 0.1370, 0.0742],
         [0.0000, 0.0131, 0.0422, 0.0140, 0.0251]],

        [[0.0104, 0.0018, 0.0018, 0.0000, 0.0026],
         [0.0451, 0.0924, 0.0000, 0.0057, 0.2784],
         [0.0008, 0.0012, 0.0020, 0.0000, 0.0140]]], grad_fn=<MulBackward0>)


In [9]:
density_1 = mask_1.mean(dim=-2)
print("density_1=", density_1)
# Something continuous that is correlated with what we want to equalize.
density_1_proxy = density_1_proxy.mean(dim=-2)
print("density_1_proxy=", density_1_proxy)
loss = (density_1_proxy * density_1).mean() * float(num_gates ** 2)
print("loss=",loss)

density_1= tensor([[0.0000, 0.0000, 0.0000, 0.6667, 0.3333],
        [0.6667, 0.3333, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3333, 0.6667, 0.0000]])
density_1_proxy= tensor([[0.1071, 0.1249, 0.1156, 0.3849, 0.2675],
        [0.4818, 0.2423, 0.1140, 0.0839, 0.0780],
        [0.0187, 0.0318, 0.1941, 0.6571, 0.0983]], grad_fn=<MeanBackward1>)
loss= tensor(2.0841, grad_fn=<MulBackward0>)


~~1. outer_experts_dim 是什么？为什么初始化的时候不传入，初始为 tuple()~~
2. loss到底是什么原理
~~3. expert_capacity是干什么的~~
4. cumsum_exclusive为什么dim向上扩充1层0，最后返回切掉代表总和的最后1层？
5. compute assignment to experts是什么作用？

In [10]:
probs = torch.zeros_like(gate_2).uniform_(0., 1.)
print("probs=",probs)
print("drop=",(probs < (gate_2 / max(0.2, 1e-9))).float().unsqueeze(-1))
mask_2 *= (probs < (gate_2 / max(0.2, 1e-9))).float().unsqueeze(-1)
print("gate_2=", gate_2)
print("mask_2",mask_2)

probs= tensor([[0.9989, 0.2530, 0.9471],
        [0.3869, 0.5935, 0.5862],
        [0.6400, 0.5997, 0.4616]])
drop= tensor([[[1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [0.]],

        [[0.],
         [1.],
         [0.]]])
gate_2= tensor([[0.2635, 0.2480, 0.2172],
        [0.1361, 0.2608, 0.0422],
        [0.0104, 0.2784, 0.0140]], grad_fn=<SqueezeBackward1>)
mask_2 tensor([[[1., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 1.]],

        [[1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 0., 0.]]])


In [25]:
position_in_expert_1 = cumsum_exclusive(mask_1, dim=-2) * mask_1
print("mask_1=",mask_1)
print("return=",cumsum_exclusive(mask_1, dim=-2))
print("position_in_expert_1",position_in_expert_1)

mask_1= tensor([[[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.]],

        [[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.]]])
return= tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 1., 0.]]])
position_in_expert_1 tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.]]])


In [21]:
t=mask_1 # [batch_size,outer_expert_size,num_gates/experts]
dim_cumsum=-2
num_dims = len(t.shape)
num_pad_dims = - dim_cumsum - 1 # 将dim变成正的，还是原来的那一维
pre_padding = (0, 0) * num_pad_dims
print("pre_padding=",pre_padding)
pre_slice   = (slice(None),) * num_pad_dims
print("pre_slice=",pre_slice)
print("F.pad(t,(*pre_padding,1,0))=",(*pre_padding, 1, 0))
padded_t = F.pad(t, (*pre_padding, 1, 0)).cumsum(dim=dim_cumsum)
print("padded_t=",padded_t)
print("slice(None,-1)=",slice(None, -1))
print("return=",padded_t[(..., slice(None, -1), *pre_slice)])

pre_padding= (0, 0)
pre_slice= (slice(None, None, None),)
F.pad(t,(*pre_padding,1,0))= (0, 0, 1, 0)
padded_t= tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 1.],
         [0., 0., 0., 2., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [2., 1., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 1., 0.],
         [0., 0., 1., 2., 0.]]])
slice(None,-1)= slice(None, -1, None)
return= tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 1., 1., 0.]]])


In [24]:
all_slice=(slice(None, -1),*pre_slice)
test_tensor=torch.randn(3,4,5)
print("all_slice=",all_slice)
print("test_tensor=",test_tensor)
print("slice=",test_tensor[(...,*all_slice)])

all_slice= (slice(None, -1, None), slice(None, None, None))
test_tensor= tensor([[[-1.4956, -0.3686,  0.0424, -0.5827,  1.0200],
         [ 1.3004,  0.6205,  0.0248,  0.2213,  0.4334],
         [-2.5851, -0.0540, -0.7103, -0.2518, -0.2877],
         [ 0.3636, -0.3669, -0.8981, -0.0413, -1.2482]],

        [[ 0.7614, -0.7035, -0.2184,  0.1117,  0.2142],
         [-1.0687,  0.2303, -2.1075,  2.2519,  0.4521],
         [-0.4342,  1.2900,  0.8189, -2.5383, -0.1784],
         [-0.6281, -0.7641, -0.8560, -0.1367,  0.8297]],

        [[ 0.5288, -0.2732,  0.0165, -0.4402,  0.1158],
         [ 0.5225, -1.8376, -0.5805,  1.7009,  0.2643],
         [-1.0354,  0.0363,  1.5778, -1.2066,  1.3709],
         [-0.8472,  1.2676,  2.6247, -1.1441, -1.2538]]])
slice= tensor([[[-1.4956, -0.3686,  0.0424, -0.5827,  1.0200],
         [ 1.3004,  0.6205,  0.0248,  0.2213,  0.4334],
         [-2.5851, -0.0540, -0.7103, -0.2518, -0.2877]],

        [[ 0.7614, -0.7035, -0.2184,  0.1117,  0.2142],
         [-1.068

In [28]:
group_size = 3
capacity_factor = 1.25
expert_capacity = min(group_size, int((group_size * capacity_factor) / num_gates))
print("expert_capacity=",expert_capacity)
expert_capacity = max(expert_capacity, MIN_EXPERT_CAPACITY)
print(expert_capacity)
expert_capacity_f = float(expert_capacity)
print(expert_capacity_f)

expert_capacity= 0
4
4.0


In [29]:
print("position_in_expert_1=",position_in_expert_1)
mask_1 *= (position_in_expert_1 < expert_capacity_f).float()
# [batch, experts]
# How many examples in this sequence go to this expert
print("mask_1=", mask_1)
mask_1_count = mask_1.sum(dim=-2, keepdim=True)
print("mask_1_count=", mask_1_count)
# [batch, group] - mostly ones, but zeros where something didn't fit
mask_1_flat = mask_1.sum(dim=-1)
print("mask_1_flat=",mask_1_flat)
# [batch, group]
position_in_expert_1 = position_in_expert_1.sum(dim=-1)
print("position_in_expert_1=",position_in_expert_1)
# Weight assigned to first expert.  [batch, group]
gate_1 *= mask_1_flat
print("gate_1=", gate_1)

position_in_expert_1= tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.]]])
mask_1= tensor([[[0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 1.],
         [0., 0., 0., 1., 0.]],

        [[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0.]],

        [[0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.]]])
mask_1_count= tensor([[[0., 0., 0., 2., 1.]],

        [[2., 1., 0., 0., 0.]],

        [[0., 0., 1., 2., 0.]]])
mask_1_flat= tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
position_in_expert_1= tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]])
gate_1= tensor([[0.4086, 0.4581, 0.6728],
        [0.5895, 0.4036, 0.9057],
        [0.9834, 0.5785, 0.9820]], grad_fn

In [2]:
aaaaa=torch.randn(2,3,4)
print(aaaaa)
a=slice(-1,None)
print(aaaaa[a])
# print(aaaaa[...,None,None])

tensor([[[-7.0308e-01, -2.0393e-02, -1.6784e+00, -1.4372e+00],
         [ 4.4806e-02, -1.4272e+00, -9.8718e-02,  1.0854e-01],
         [-9.0818e-01,  4.7623e-01,  1.2458e-03, -1.1817e+00]],

        [[-7.9067e-01,  1.2508e+00,  1.8369e+00, -6.5970e-01],
         [-1.9559e+00,  5.2007e-01, -5.7840e-01, -6.8399e-01],
         [ 1.3251e+00, -8.9617e-02,  1.2529e+00, -1.4444e+00]]])
tensor([[[-0.7907,  1.2508,  1.8369, -0.6597],
         [-1.9559,  0.5201, -0.5784, -0.6840],
         [ 1.3251, -0.0896,  1.2529, -1.4444]]])
